In [1]:
import numpy as np # linear algebra
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
# Cell 1: Install required packages
!pip install -q transformers accelerate bitsandbytes langchain langchain_community sentencepiece einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB

In [3]:
# Cell 2:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
import torch
import re
import json

# Verify environment
print(f"PyTorch version: {torch.__version__}")
print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

2025-06-12 07:09:33.262210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749712173.460154      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749712173.515141      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


PyTorch version: 2.6.0+cu124
GPU available: True
GPU name: Tesla P100-PCIE-16GB


In [4]:
# Cell 3:
model_name = "Qwen/Qwen3-4B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    pad_token='<|endoftext|>',
    padding_side='left'
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# Create separate pipelines for different tasks
info_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.3,  
    top_p=0.9,
    repetition_penalty=1.3,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)

question_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  
    temperature=1,    
    top_p=0.95,
    repetition_penalty=1.3,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)

info_llm = HuggingFacePipeline(pipeline=info_pipeline)
question_llm = HuggingFacePipeline(pipeline=question_pipeline)

tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
Device set to use cuda:0
/tmp/ipykernel_35/2102704115.py:41: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  info_llm = HuggingFacePipeline(pipeline=info_pipeline)


In [5]:
# Cell 4:
# Updated prompt templates
system_prompt = """<|im_start|>system
You are TalentBot, an AI hiring assistant for TalentScout recruitment agency. Your job is asking technical question to the candidate and collect their answers.
Your objectives are:
1. Professionally collect candidate information.
2. Generate high-quality, meaningful and complete technical questions.
3. Maintain context and a coherent conversational flow with candidate.
4. Never deviate from your purpose of assisting with candidate screening.
5. Carefully resolve candidate's query during screening test. First figure out what is the candidate query and context along that.
6. When evaluating or presenting code snippets, ensure the code is complete, syntactically correct, and logically sound.
7. Promptly and accurately address any clarifications or queries the candidate raises about the assessment itself, first understanding their question fully before responding.
Current conversation stage: {stage}
{history}<|im_end|>
"""

info_gathering_template = system_prompt + """<|im_start|>user
{input}<|im_end|>
<|im_start|>assistant
"""

tech_question_template = """<|im_start|>system
You are TalentBot, the technical interviewer for a candidate. The candidate is applying for the position of {desired_position} and has {years_experience} years of experience. The candidate's technology stack includes: {tech_stack}.

Your task is to generate exactly 5 technical interview questions tailored to this candidate. Use the information given to ensure the questions are relevant and appropriately challenging. Follow these guidelines:

- Ask exactly five questions, each on a separate line prefixed with its number (1. to 5.).
- Tailor each question specifically to the technologies listed and the desired position.
- Match the difficulty level to the candidate's experience ({years_experience} years).
- Begin with more fundamental concepts and progressively increase in complexity.
- Include a variety of question types.
- Ensure each question is standalone, actionable, and non-overlapping in scope.
- Avoid questions about topics not included in the candidate's tech stack.
- Use clear and concise professional language appropriate for an interview.
- Do NOT provide any answers or hints to the questions — only ask the questions.
- If the candidate asks for clarification on a question, clarify the question without giving away any answer.
- Ensure each question is specific and actionable, not overly broad or vague.
- Focus on problem-solving approach, code implementation details, and understanding of the technologies.
- Include questions that evaluate both theoretical understanding and hands-on skills.
- Ensure no two questions are redundant or too similar.
- Check that all five questions are included before finalizing. Ensure that they are valid and complete.
- Make sure questions are clear and concise
- Verify completeness and clarity of all five questions before delivering.
<|im_end|>
<|im_start|>assistant
1.
"""

closing_template = system_prompt + """<|im_start|>user
{input}<|im_end|>
<|im_start|>assistant
"""

# Create prompt templates
info_gathering_prompt = PromptTemplate(
    input_variables=["stage", "history", "input"],
    template=info_gathering_template
)

tech_question_prompt = PromptTemplate(
    input_variables=["tech_stack", "years_experience", "desired_position"],
    template=tech_question_template
)

closing_prompt = PromptTemplate(
    input_variables=["stage", "history", "input"],
    template=closing_template
)

# Create chains
info_gathering_chain = LLMChain(
    llm=info_llm,
    prompt=info_gathering_prompt,
    verbose=False
)

tech_question_chain = LLMChain(
    llm=question_llm,
    prompt=tech_question_prompt,
    verbose=False
)

closing_chain = LLMChain(
    llm=info_llm,
    prompt=closing_prompt,
    verbose=False
)


/tmp/ipykernel_35/1514396138.py:71: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  info_gathering_chain = LLMChain(


In [6]:
# New chains for query handling
relevance_prompt = PromptTemplate(
    input_variables=["current_question", "query_text"],
    template="""<|im_start|>system
You are an expert interviewer assistant with a focus on evaluating the relevance of candidate queries in a technical interview context. For each pair of inputs, perform two checks:

Relevance: Determine whether the Candidate Query is directly related to the Current Question.

Question Quality: Assess whether the Current Question is well-formed—meaning it is correct, complete, and answerable as stated.

Respond with exactly one of the following, with no additional text:

- yes and question is correct

- yes and question is incorrect

- no

Current Question: {current_question}
Candidate Query: {query_text}
<|im_end|>
<|im_start|>assistant
"""
)

revision_prompt = PromptTemplate(
    input_variables=["current_question", "query_text"],
    template="""
    <|im_start|>system
You are a senior technical interviewer and expert question writer. A candidate has requested clarification on the following interview question:
{current_question}

They have raised this specific query for additional context or detail:
{query_text}

Your task is to deliver a revised version of the original question that:
1. Preserves the original technical objectives and scope.
2. Maintains the intended difficulty and challenge level.
3. Enhances clarity, precision, and completeness.
4. Employs a professional, engaging tone suitable for an interview setting.

Guidelines:
- If the candidate’s query highlights missing context, code examples, or parameters, integrate them directly into the revised question.
- Do not include any commentary, explanation, or notes—return only the updated question.
- Ensure the question remains actionable, unambiguous, and aligned with its original intent.
<|im_end|>
<|im_start|>assistant

"""
)

# Create query handling chains
relevance_chain = LLMChain(
    llm=info_llm,
    prompt=relevance_prompt,
    verbose=False
)

revision_chain = LLMChain(
    llm=question_llm,
    prompt=revision_prompt,
    verbose=False
)

In [7]:
# Cell 5:
class HiringState:
    FIELDS = [
        "full_name",
        "email",
        "phone",
        "years_experience",
        "desired_position",
        "current_location"
    ]
    FIELD_DISPLAY = {
        "full_name": "full name",
        "email": "email address",
        "phone": "phone number",
        "years_experience": "years of professional experience",
        "desired_position": "desired position",
        "current_location": "current location",
        "tech_stack": "tech stack (comma-separated list of technologies)"
    }
    FIELD_PROMPTS = {
        "full_name": "What is your full name?",
        "email": "What is your email address?",
        "phone": "What is your phone number?",
        "years_experience": "How many years of professional experience do you have?",
        "desired_position": "What is your desired position?",
        "current_location": "What is your current location?",
        "tech_stack": "Please list your technical skills (comma-separated):"
    }
    FIELD_VALIDATORS = {
        "email": lambda x: re.match(r"[^@]+@[^@]+\.[^@]+", x) is not None,
        "phone": lambda x: re.match(r"^\+?[0-9\s\-\(\)]{7,}$", x) is not None,
        "years_experience": lambda x: re.match(r"^\d+$", x) is not None
    }
    FIELD_ERRORS = {
        "email": "Please enter a valid email address (e.g., name@example.com).",
        "phone": "Please enter a valid phone number (e.g., +1 123-456-7890).",
        "years_experience": "Please enter a valid number of years (e.g., 5)."
    }

    def __init__(self):
        self.reset()

    def reset(self):
        self.stage = "greeting"
        self.current_field_idx = 0
        self.candidate_data = {field: None for field in self.FIELDS}
        # Ensure tech_stack key exists in candidate_data
        self.candidate_data["tech_stack"] = None
        self.tech_questions = []
        self.current_question_idx = 0
        self.memory = ConversationBufferMemory()
        self.conversation_log = []

    def get_current_field(self):
        if self.current_field_idx < len(self.FIELDS):
            return self.FIELDS[self.current_field_idx]
        return None

    def record_response(self, field, value):
        if field in self.FIELDS:
            self.candidate_data[field] = value
            self.log_interaction(f"User provided {field}: {value}")

    def next_field(self):
        if self.current_field_idx < len(self.FIELDS) - 1:
            self.current_field_idx += 1
            return True
        return False

    def log_interaction(self, message):
        self.conversation_log.append(message)

    def to_dict(self):
        return {
            "stage": self.stage,
            "current_field": self.get_current_field(),
            "candidate_data": self.candidate_data,
            "tech_questions": self.tech_questions,
            "current_question_idx": self.current_question_idx
        }

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    truncation=True
)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
# cell 6:
def clean_response(response):
    """Clean and format the model response"""
    # Remove special tokens
    response = re.sub(r'<\|im_start\|>.*?<\|im_end\|>', '', response, flags=re.DOTALL)
    response = re.sub(r'<\|.*?\|>', '', response, flags=re.DOTALL)
    # Remove thinking tags and content
    response = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL)
    if "assistant" in response.lower():
        parts = re.split(r'assistant', response, flags=re.IGNORECASE)
        if len(parts) > 1:
            response = parts[-1].strip()
    response = re.sub(r'^\d+[\.\)\s]*', '', response).strip()
    response = re.sub(r'\n+', '\n', response).strip()
    return response

def extract_questions(response):
    """Extract generated questions from model response"""
    questions = []
    lines = response.split('\n')
    for line in lines:
        match = re.match(r'(\d+)\.?\s*(.*)', line.strip())
        if match:
            question_num = int(match.group(1))
            question_text = match.group(2).strip()
            if len(question_text) > 20 and '?' in question_text:
                questions.append(f"{question_num}. {question_text}")
    if len(questions) < 5:
        pattern = r'\d+\.\s*([^\n?]+\??)'
        alt_questions = re.findall(pattern, response)
        questions = [f"{i+1}. {q.strip()}" for i, q in enumerate(alt_questions[:5])]
    return questions[:5]

def format_tech_stack(tech_input):
    return [tech.strip() for tech in tech_input.split(',') if tech.strip()]

def analyze_sentiment(text):
    if len(text) < 3:
        return {"label": "NEUTRAL", "score": 0.0}
    try:
        result = sentiment_pipeline(text, truncation=True)[0]
        return {"label": result['label'], "score": float(result['score'])}
    except:
        return {"label": "ERROR", "score": 0.0}

def print_state(state):
    """Debug function to print current state"""
    print("\n" + "="*50)
    print(f"Current Stage: {state.stage}")
    print(f"Current Field: {state.get_current_field()}")
    print(f"Questions: {len(state.tech_questions)}")
    print(f"Data: {json.dumps(state.candidate_data, indent=2)}")
    print("="*50 + "\n")

def generate_tech_questions(state):
    """Generate technical questions with better prompting and validation"""
    required_fields = ['tech_stack', 'years_experience', 'desired_position']
    if any(state.candidate_data[field] is None for field in required_fields):
        state.stage = "closing"
        return "We don't have enough information to generate questions. Thank you for your time.", state

    response = tech_question_chain.invoke({
        "tech_stack": ", ".join(state.candidate_data["tech_stack"]),
        "years_experience": state.candidate_data["years_experience"],
        "desired_position": state.candidate_data["desired_position"]
    })['text']

    cleaned_response = clean_response(response)
    state.tech_questions = extract_questions(cleaned_response)

    if not state.tech_questions:
        position = state.candidate_data["desired_position"] or "this position"
        tech = state.candidate_data["tech_stack"][0] if state.candidate_data["tech_stack"] else "your primary technology"
        state.tech_questions = [
            "1. What experience do you have with " + tech + "?",
            "2. Describe a challenging project you've worked on using " + tech + ".",
            "3. How would you debug a performance issue in a " + tech + " application?",
            "4. What best practices do you follow when working with " + tech + "?",
            "5. How does your experience align with the requirements for " + position + "?"
        ]

    state.log_interaction(f"Generated {len(state.tech_questions)} technical questions")
    return None, state

In [9]:
def handle_conversation(user_input, state):
    # Exit condition
    if user_input.lower() in ['exit', 'quit', 'stop', 'end']:
        closing_response = "Thank you for your time. We'll be in touch soon!"
        state.memory.save_context({"input": user_input}, {"output": closing_response})
        state.log_interaction(f"Assistant: {closing_response}")
        return closing_response, state

    if user_input:
        state.log_interaction(f"User: {user_input}")
        if state.stage not in ["greeting", "awaiting_start", "info_gathering"]:
            sentiment = analyze_sentiment(user_input)
            state.log_interaction(f"Sentiment: {sentiment['label']} ({sentiment['score']:.2f})")

    state.memory.save_context({"input": user_input}, {"output": ""})

    # Handle query requests during technical interview
    if state.stage == "technical_interview" and user_input.lower().startswith("query:"):
        query_text = user_input[len("query:"):].strip()
        current_question = state.tech_questions[state.current_question_idx]
        
        # Check relevance of query
        relevance_response = relevance_chain.invoke({
            "current_question": current_question,
            "query_text": query_text
        })['text'].lower().strip()
        
        if "yes" in relevance_response:
            # Generate revised question
            revised_question = revision_chain.invoke({
                "current_question": current_question,
                "query_text": query_text
            })['text'].strip()
            
            # Clean and store revised question
            revised_question = clean_response(revised_question)
            state.tech_questions[state.current_question_idx] = revised_question
            state.log_interaction(f"Revised Question: {revised_question}")
            
            response = f"Thank you for your query. Here's the revised question:\n\n{revised_question}"
        else:
            response = "Your query doesn't appear relevant to the current question. Please answer the original question."
        
        state.memory.save_context({"input": user_input}, {"output": response})
        state.log_interaction(f"Assistant: {response}")
        return response, state

    # Existing conversation flow
    if state.stage == "greeting":
        greeting = "Hello! I'm TalentBot from TalentScout. I'll guide you through our initial screening process. "\
                   "If you're ready to begin, please type 'Start'.\n\n"\
                   "You can type 'exit' at any time to end the conversation.\n"\
                   "During technical questions, you can request clarification by typing 'query: your question'."
        state.stage = "awaiting_start"
        state.log_interaction("System: Initial greeting")
        return greeting, state

    elif state.stage == "awaiting_start":
        if user_input.lower() == "start":
            state.stage = "info_gathering"
            state.log_interaction("User started the process")
            first_field = state.get_current_field()
            return state.FIELD_PROMPTS[first_field], state
        else:
            return "Please type 'Start' when you're ready to begin the screening process.", state

    elif state.stage == "info_gathering":
        current_field = state.get_current_field()
        if current_field in state.FIELD_VALIDATORS:
            validator = state.FIELD_VALIDATORS[current_field]
            if not validator(user_input):
                return state.FIELD_ERRORS[current_field], state
        state.record_response(current_field, user_input)
        if state.next_field():
            next_field = state.get_current_field()
            return state.FIELD_PROMPTS[next_field], state
        else:
            state.stage = "tech_stack_collection"
            state.log_interaction("System: Collecting tech stack")
            return state.FIELD_PROMPTS["tech_stack"], state

    elif state.stage == "tech_stack_collection":
        tech_stack = format_tech_stack(user_input)
        state.candidate_data["tech_stack"] = tech_stack
        state.log_interaction(f"Tech stack: {', '.join(tech_stack)}")
        state.stage = "technical_interview"
        msg, state = generate_tech_questions(state)
        if msg:
            return msg, state
        if state.tech_questions:
            first_question = state.tech_questions[0].split('.', 1)[1].strip()
            response = f"Thank you! Let's begin the technical assessment. You'll be asked {len(state.tech_questions)} questions.\n\n"\
                       f"Question 1: {first_question}\n\n"\
                       "If you need clarification on any question, type 'query: your question'."
        else:
            state.stage = "closing"
            response = "We've completed the initial screening. Thank you for your time!"
        state.memory.save_context({"input": user_input}, {"output": response})
        state.log_interaction(f"Assistant: {response}")
        return response, state

    elif state.stage == "technical_interview":
        if state.current_question_idx < len(state.tech_questions):
            question = state.tech_questions[state.current_question_idx]
            state.log_interaction(f"Question {state.current_question_idx+1}: {question}")
            
            # Only log as answer if it's not a query request
            if not user_input.lower().startswith("query:"):
                state.log_interaction(f"Answer: {user_input[:200]}...")
                state.current_question_idx += 1
            
            if state.current_question_idx < len(state.tech_questions):
                next_q = state.tech_questions[state.current_question_idx].split('.', 1)[1].strip()
                response = f"Question {state.current_question_idx + 1}: {next_q}"
            else:
                state.stage = "closing"
                state.log_interaction("Completed technical assessment")
                response = "Thank you for completing the assessment! Our team will review your answers and contact you soon."
            
            state.memory.save_context({"input": user_input}, {"output": response})
            state.log_interaction(f"Assistant: {response}")
            return response, state

    if state.stage == "closing":
        response = "Thank you again! Our team will review your application shortly."
        state.memory.save_context({"input": user_input}, {"output": response})
        state.log_interaction(f"Assistant: {response}")
        return response, state

    response = "I'm here to assist with your job application. Could you please rephrase that?"
    state.memory.save_context({"input": user_input}, {"output": response})
    state.log_interaction(f"Assistant: {response}")
    return response, state

In [17]:
# Cell 9:
def run_conversation():
    state = HiringState()
    print("\n" + "="*50)
    print("TalentScout Hiring Assistant")
    print("Type 'exit' at any time to end the conversation")
    print("="*50 + "\n")

    response, state = handle_conversation("", state)
    print(f"Assistant: {response}")

    while True:
        user_input = input("Candidate: ")
        if user_input.lower() in ['exit', 'quit', 'stop', 'end']:
            print("\nAssistant: Thank you for your time. The conversation has ended.")
            break
        response, state = handle_conversation(user_input, state)
        print(f"\nAssistant: {response}")
        if state.stage == "closing" and "thank" in response.lower():
            break

    print("\n===== Conversation Summary =====")
    print(f"Total Interactions: {len(state.conversation_log)}")

    sentiment_counts = {"POSITIVE": 0, "NEGATIVE": 0, "NEUTRAL": 0}
    sentiment_scores = []
    for log in state.conversation_log:
        if log.startswith("Sentiment:"):
            parts = log.split(':')
            if len(parts) > 1:
                sentiment = parts[1].strip().split()[0]
                if sentiment in sentiment_counts:
                    sentiment_counts[sentiment] += 1
                if '(' in parts[1]:
                    score_str = parts[1].split('(')[1].split(')')[0]
                    try:
                        sentiment_scores.append(float(score_str))
                    except:
                        pass

    if sentiment_scores:
        avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)
        print(f"\nSentiment Analysis:")
        print(f"- Positive Responses: {sentiment_counts['POSITIVE']}")
        print(f"- Negative Responses: {sentiment_counts['NEGATIVE']}")
        print(f"- Neutral Responses: {sentiment_counts['NEUTRAL']}")
        print(f"- Average Sentiment Score: {avg_sentiment:.2f}")
        print(f"- Overall Tone: {'Positive' if avg_sentiment > 0.6 else 'Negative' if avg_sentiment < 0.4 else 'Neutral'}")

    

# Start conversation
run_conversation()


TalentScout Hiring Assistant
Type 'exit' at any time to end the conversation

Assistant: Hello! I'm TalentBot from TalentScout. I'll guide you through our initial screening process. If you're ready to begin, please type 'Start'.

You can type 'exit' at any time to end the conversation.
During technical questions, you can request clarification by typing 'query: your question'.


Candidate:  Start



Assistant: What is your full name?


Candidate:  alpha



Assistant: What is your email address?


...


Assistant: Question 5: What strategies would you employ to optimize the inference speed of a trained PyTorch model deployed in production, considering factors such as GPU utilization, quantization techniques, and模型 pruning methods?


Candidate:  query: please clarify the question



Assistant: Thank you for your query. Here's the revised question:

How would you approach optimizing the inference latency of a large-scale deep learning model implemented in PyTorch for real-time applications, while balancing computational efficiency and maintaining acceptable prediction accuracy? Specifically, outline your strategy for leveraging GPU acceleration through framework-specific optimizations, applying state-of-the-art quantization techniques (including but not limited to post-training static量化、dynamic量化, and mixed-precision training), and implementing effective model pruning methods (such as magnitude-based pruning, channel pruning, or structural pruning) tailored to different neural network architectures. Additionally, explain how you would evaluate the impact of these optimizations on overall throughput, memory footprint, and energy consumption during deployment.


Candidate:  next



Assistant: Thank you for completing the assessment! Our team will review your answers and contact you soon.

===== Conversation Summary =====
Total Interactions: 51

Sentiment Analysis:
- Positive Responses: 6
- Negative Responses: 1
- Neutral Responses: 0
- Average Sentiment Score: 0.92
- Overall Tone: Positive
